## CNN 모델 만들기 <hr>

- CNN 모델 전이학습: Mobilenet

1. 모듈 로딩

In [27]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torchvision.datasets import ImageFolder
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset

In [28]:
#전처리/ Resizing, Tensorize
preprocessing = transforms.Compose([transforms.Resize((32,32)), transforms.ToTensor()])

In [29]:
# 폴더명 = 분류 클래스
root = 'TrashType_Image_Dataset'
dataset = ImageFolder(root = root, transform= preprocessing) #전처리
loader = DataLoader(dataset = dataset, batch_size=32, shuffle=False) #dataloader

In [30]:
# train - test split
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size

In [31]:
# mobilenet model

model = models.mobilenet_v2(pretrained = True)

c:\Users\kdp\anaconda3\envs\NLP_38\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\kdp\anaconda3\envs\NLP_38\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [32]:
num_features = model.classifier[1].in_features
model.classifier = nn.Sequential(nn.Dropout(0.2),
                                  nn.Linear(num_features, 1),# 이진분류니까 출력 노드는 한 개
                                  nn.Sigmoid())

In [33]:
# optimizer, 손실함수 정의하기

creterion = nn.BCEWithLogitsLoss()
optimizer = optim.AdamW(model.parameters(), lr=0.01, weight_decay=1e-5)

In [34]:
# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=

In [35]:
#model training
num_epochs = 150
for ep in range(num_epochs):
    model.train()
    running_loss = 0.0

    for inputs, labels in loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        
        outputs = model(inputs).squeeze()
        loss = creterion(outputs, labels.float())

        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(loader.dataset)
    print(f'[EPOCH] {ep+1} / {num_epochs}\n\n[LOSS] : {epoch_loss:.4f}')

[EPOCH] 1 / 150

[LOSS] : 0.6954
[EPOCH] 2 / 150

[LOSS] : 0.6931
[EPOCH] 3 / 150

[LOSS] : 0.6931
[EPOCH] 4 / 150

[LOSS] : 0.6931
[EPOCH] 5 / 150

[LOSS] : 0.6931
[EPOCH] 6 / 150

[LOSS] : 0.6931
[EPOCH] 7 / 150

[LOSS] : 0.6931
[EPOCH] 8 / 150

[LOSS] : 0.6455
[EPOCH] 9 / 150

[LOSS] : 0.7341
[EPOCH] 10 / 150

[LOSS] : 0.6931
[EPOCH] 11 / 150

[LOSS] : 0.6931
[EPOCH] 12 / 150

[LOSS] : 0.6931
[EPOCH] 13 / 150

[LOSS] : 0.6931
[EPOCH] 14 / 150

[LOSS] : 0.6931
[EPOCH] 15 / 150

[LOSS] : 0.6931
[EPOCH] 16 / 150

[LOSS] : 0.6931
[EPOCH] 17 / 150

[LOSS] : 0.6931
[EPOCH] 18 / 150

[LOSS] : 0.6931
[EPOCH] 19 / 150

[LOSS] : 0.6931
[EPOCH] 20 / 150

[LOSS] : 0.6931
[EPOCH] 21 / 150

[LOSS] : 0.6931
[EPOCH] 22 / 150

[LOSS] : 0.6931
[EPOCH] 23 / 150

[LOSS] : 0.6931
[EPOCH] 24 / 150

[LOSS] : 0.6931
[EPOCH] 25 / 150

[LOSS] : 0.6931
[EPOCH] 26 / 150

[LOSS] : 0.6931
[EPOCH] 27 / 150

[LOSS] : 0.6931
[EPOCH] 28 / 150

[LOSS] : 0.6931
[EPOCH] 29 / 150

[LOSS] : 0.6931
[EPOCH] 30 / 150

[LOSS

In [36]:
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score
import numpy as np

In [37]:
model.eval()
true_labels = []
pred_probs = []

with torch.no_grad():
    for inputs, labels in loader:
        inputs, labels = inputs.to(device), labels.to(device)

        #model evaluation
        outputs = model(inputs).squeeze()
        probs = torch.sigmoid(outputs)
        preds = (probs > 0.5).float() # 확률 기반 이진분류

        true_labels.extend(labels.cpu().numpy())
        pred_probs.extend(probs.cpu().numpy())

preds = (np.array(pred_probs) > 0.5).astype(int)
# 정확도 계산
accuracy = accuracy_score(true_labels, preds)
print(f'Acc: {accuracy * 100:.2f}%')

#roc auc 계산
auc = roc_auc_score(true_labels, preds)
print(f'ROC_AUC {auc:.4f}')

#classification report
print(classification_report(true_labels, preds, target_names=['0','1']))

Acc: 52.88%
ROC_AUC 0.5000
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      2076
           1       0.53      1.00      0.69      2330

    accuracy                           0.53      4406
   macro avg       0.26      0.50      0.35      4406
weighted avg       0.28      0.53      0.37      4406



c:\Users\kdp\anaconda3\envs\NLP_38\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\kdp\anaconda3\envs\NLP_38\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\kdp\anaconda3\envs\NLP_38\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [38]:
torch.save(model.state_dict(),'Model2.pth' )